In [ ]:
import pandas as pd
import warnings
from my_analysis_toolkit import read_encrypted_excels, intake_validate
from my_static_data import tt_passwords, intake_columns, intake_basic_columns, intake_date_columns,victimhood

In [ ]:
# Avoid representing large numbers in scientific form. To reset, use the commented line.
pd.set_option('display.float_format', '{:.1f}'.format)
# pd.reset_option('display.float_format')

# Display maximum column width:
pd.set_option('display.max_colwidth', None)

# Suppress openpyxl data validation warning
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [ ]:
passwords = tt_passwords
tracking_tools = read_encrypted_excels(r'000', passwords)

# Process Intake Sheets

In [ ]:
sheet = 'Int'
sp = 'ntsp'
# columns_names = psfs_columns
# basic_columns = psfs_basic_columns
# date_columns = psfs_date_columns
# int_columns = psfs_int_columns tt_passwords

In [ ]:
yq = tracking_tools['tt_pt_HR_v04.xlsx'][sheet]
yq

In [ ]:
yq.columns = intake_columns_names
yq = yq.drop(yq.index[0]).reset_index(drop=True)
yq.dropna(subset=intake_basic_columns, how='all', inplace=True)
yq.insert(0, sp, 'YQ')

In [ ]:
sa = tracking_tools['tt_psc_SA_v04.xlsx'][sheet]

In [ ]:
sa.columns = intake_columns_names
sa = sa.drop(sa.index[0]).reset_index(drop=True)
sa.dropna(subset=intake_basic_columns, how='all', inplace=True)
sa.insert(0, sp, 'SA')

In [ ]:
ij = tracking_tools['tt_psc_IJ_v04.xlsx'][sheet]

In [ ]:
ij.columns = intake_columns_names
ij = ij.drop(ij.index[0]).reset_index(drop=True)
ij.dropna(subset=intake_basic_columns, how='all', inplace=True)
ij.insert(0, sp, 'IJ')

In [ ]:
la = tracking_tools['tt_psc_LA_v04.xlsx'][sheet]

In [ ]:
la.columns = intake_columns_names
la = la.drop(la.index[0]).reset_index(drop=True)
la.dropna(subset=intake_basic_columns, how='all', inplace=True)
la.insert(0, sp, 'LA')

In [ ]:
mt = tracking_tools['tt_psc_MT_v04.xlsx'][sheet]

In [ ]:
mt.columns = intake_columns_names
mt = mt.drop(mt.index[0]).reset_index(drop=True)
mt.dropna(subset=intake_basic_columns, how='all', inplace=True)
mt.insert(0, sp, 'MT')

In [ ]:
intake_rows = mt.shape[0] + yq.shape[0] + sa.shape[0] + ij.shape[0] + la.shape[0]
intake_rows

In [ ]:
intake = pd.concat([yq, sa, ij, la, mt], ignore_index=True)

In [ ]:
# All intake csv file
intake.to_csv(r'001/0001_ps_nt_092300.csv', index=False)

# Intake Dataframe

In [ ]:
ntk = pd.read_csv(r'001/0001_ps_nt_092300.csv')
ntk = ntk.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [ ]:
ntk.shape

In [ ]:
ntk.info()

In [ ]:
ntk

In [ ]:
ntk['fcid'] = ntk['fcid'].astype('Int64')
ntk[intake_date_columns] = ntk[intake_date_columns].apply(pd.to_datetime)

In [ ]:
# First cleaned csv file
ntk.to_csv(r'010/0001_ps_nt_092301.csv', index=False)

# Intake Analysis Version

In [ ]:
nt = pd.read_csv(r'010/0001_ps_nt_092301.csv')

In [ ]:
nt[intake_date_columns] = nt[intake_date_columns].apply(pd.to_datetime)
nt['fcid'] = nt['fcid'].astype('Int64')


In [ ]:
nt.info()

In [ ]:
nt

In [ ]:
# Encoding binary variables - victimhood
nt[victimhood] = nt[victimhood].replace({
    'Yes': 1,
    'No': 0
}).astype('Int64')

In [ ]:
if 'vict' not in nt.columns:
    nt.insert(22, 'vict', nt[victimhood].sum(axis=1))
# nt['vict'] = nt[victimhood].sum(axis=1)

In [ ]:
# Checking if there are duplicated records.
# nt[nt['fcid'].duplicated(keep=False)]
# No duplicated records found unto September 2023 data.

## Checking intake session validity sessions date.
- -1: session 1 exists only.
- 1: session 1, 2 exist and 2 > 1
- 2: session 1, 2, exist and 3 > 2 > 1
- 3: session 1, 2, 3 and/or 4 exist and 4 > (3 > 2) > 1
- 0: There is and issue in the dates validity

In [ ]:
# Checking the validity of the intake sessions dates using intake_validity function.
if 'nt' not in nt.columns:
    nt.insert(
        6,
        'nt',
        nt.apply(
            lambda  row: intake_validate(
                row['nt1'],
                row['nt2'],
                row['nt3'],
                row['ntre']
            ),
            axis=1
        )
    )

In [ ]:
# Intake Analysis Complete Dataframe
nt.to_csv(r'010/0001_ps_nt_092302.csv', index=False)

In [ ]:
# Intake Analysis Basic Dataframe
basic_nt = nt.copy()
basic_nt = basic_nt.drop(columns=['firstname', 'lastname', 'note'])
basic_nt.to_csv(r'011/0001_ps_nt_092310.csv', index=False)

### INTAKE ASSESSMENT DATA IS PREPARED AND READY